### Data Preparation

In [0]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

pd.options.mode.chained_assignment = None

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My\ Drive/Datahack\ NLP\ Workshop/HotelReviews/

7282_1.csv		      Datafiniti_Hotel_Reviews_Jun19.csv
Datafiniti_Hotel_Reviews.csv  hotel-reviews.zip


In [0]:
data_path = "/content/drive/My Drive/Datahack NLP Workshop/HotelReviews/"
df = pd.read_csv(data_path + "Datafiniti_Hotel_Reviews.csv")
df.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2013-11-14T00:00:00Z,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2014-07-06T00:00:00Z,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2015-01-02T00:00:00Z,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-05-15T00:00:00Z,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-07-09T00:00:00Z,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


### Lower Casing

In [0]:
def lower_casing(text):
    return str(text).lower()

df["text_preprocessed"] = df["reviews.text"].apply(lambda x: lower_casing(x))
df[["reviews.text", "text_preprocessed"]].head()

,reviews.text,text_preprocessed
0,Our experience at Rancho Valencia was absolute...,our experience at rancho valencia was absolute...
1,Amazing place. Everyone was extremely warm and...,amazing place. everyone was extremely warm and...
2,We booked a 3 night stay at Rancho Valencia to...,we booked a 3 night stay at rancho valencia to...
3,Currently in bed writing this for the past hr ...,currently in bed writing this for the past hr ...
4,I live in Md and the Aloft is my Home away fro...,i live in md and the aloft is my home away fro...


### Punctuation Removal

In [0]:
# https://www.programiz.com/python-programming/methods/string/translate
PUNCT_TO_REMOVE = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~' #string.punctuation
def remove_punctuation(text):
    """function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

print("Before: ", df["text_preprocessed"][0], "\n")
df["text_preprocessed"] = df["text_preprocessed"].apply(lambda x: remove_punctuation(x))
print("After: ", df["text_preprocessed"][0])

Before:  our experience at rancho valencia was absolutely perfect from beginning to end!!!! we felt special and very happy during our stayed. i would come back in a heart beat!!! 

After:  our experience at rancho valencia was absolutely perfect from beginning to end we felt special and very happy during our stayed. i would come back in a heart beat


### Stopword Removal

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

print("Before: ", df["text_preprocessed"][0], "\n")
df["text_preprocessed"] = df["text_preprocessed"].apply(lambda x: remove_stopwords(x))
print("After: ", df["text_preprocessed"][0])

Before:  our experience at rancho valencia was absolutely perfect from beginning to end we felt special and very happy during our stayed. i would come back in a heart beat 

After:  experience rancho valencia absolutely perfect beginning end felt special happy stayed. would come back heart beat


### Stemming

In [0]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

print("Before: ", df["text_preprocessed"][0], "\n")
df["text_preprocessed"] = df["text_preprocessed"].apply(lambda x: stem_words(x))
print("After: ", df["text_preprocessed"][0])

Before:  experience rancho valencia absolutely perfect beginning end felt special happy stayed. would come back heart beat 

After:  experi rancho valencia absolut perfect begin end felt special happi stayed. would come back heart beat


### Lemmatization

In [0]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(str(text).split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN))
                     for word, pos in pos_tagged_text])

print("Before: ", df["reviews.text"][0], "\n")
df["text_preprocessed"] = df["reviews.text"].apply(lambda x: lemmatize_words(x))
print("After: ", df["text_preprocessed"][0])

Before:  Our experience at Rancho Valencia was absolutely perfect from beginning to end!!!! We felt special and very happy during our stayed. I would come back in a heart beat!!! 

After:  Our experience at Rancho Valencia be absolutely perfect from begin to end!!!! We felt special and very happy during our stayed. I would come back in a heart beat!!!


### Handling Emojis

In [0]:
def remove_emoji(string):
  emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)
remove_emoji("game is on 🔥")

'game is on '

In [0]:
data_path = "/content/drive/My Drive/Datahack NLP Workshop/HelperCodes/"
import sys
sys.path.append(data_path)

from text_prep_config import UNICODE_EMO

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text

text = "game is on 🔥"
convert_emojis(text)

'game is on fire'

### Handling Emoticons

In [0]:
from text_prep_config import EMOTICONS
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

remove_emoticons("Hello :-)")

'Hello '

In [0]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

text = "Hello :-) :-)"
convert_emoticons(text)

'Hello Happy_face_smiley Happy_face_smiley'

### URL Removal

In [0]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

'Driverless AI NLP blog post on '

### HTML Removal

In [0]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

print(remove_html(text))


 H2O
 AutoML
 Driverless AI



### Handling Contractions

In [0]:
# don't to do not
from text_prep_config import CONTRACTIONS_DICT

contraction_words = set(CONTRACTIONS_DICT.keys())
def expand_contractions(text):
    new_text = []
    for w in str(text).split():
        if w in contraction_words:
            new_text.append(CONTRACTIONS_DICT[w])
        else:
            new_text.append(w)
    return " ".join(new_text)

expand_contractions("I'm going to DHS")

'I am going to DHS'

### Handling Chatwords

In [0]:
from text_prep_config import CHAT_WORDS_DICT

chat_words = set(CHAT_WORDS_DICT.keys())
def expand_contractions(text):
    new_text = []
    for w in str(text).split():
        if w in chat_words:
            new_text.append(CHAT_WORDS_DICT[w])
        else:
            new_text.append(w)
    return " ".join(new_text)

expand_contractions("IMHO this is possible")

'In My Honest/Humble Opinion this is possible'

### Duplication Removal

In [0]:
#### Removal of duplicate characters
text = "howdyyy!!! how are you doingggg????"

#Write your code to get the unique characters
def removeDupWithoutOrder(str):
    return "".join(set(str))

removeDupWithoutOrder(text)

'hgrnu!a?woi yed'

In [0]:
#Write your code to get all the unique characters in same order
from collections import OrderedDict 
def removeDupWithOrder(str): 
    return "".join(OrderedDict.fromkeys(str))  

removeDupWithOrder(text)

'howdy! areuing?'

In [0]:
## Write your code to replace those duplicate characters that occur more than once in chat format
sent = []
for char in text.split():
  char = removeDupWithOrder(char)
  sent.append(char)
sent = " ".join(sent)
print(sent)

howdy! how are you doing?
